In [62]:
cd /Users/tedalcorn/desktop/2024\ 07\ The\ Trace\ -\ Baltimore\ Bar\ Bill/2024\ 07\ 05\ Mapping

/Users/tedalcorn/Desktop/2024 07 The Trace - Baltimore Bar Bill/2024 07 05 Mapping


/Users/tedalcorn/.pyenv/versions/3.11.8/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [63]:
import csv
import requests
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import time
import re

In [64]:
with open('240520_Baltimore_master.csv', newline='') as fp:
    reader = csv.DictReader(fp)
    data = list(reader)

# Display the first three items
print(data[:3])

[{'\ufeffBlock': '16-Dec', 'Street': '20TH STREET WEST', 'Address': '12-16 20TH STREET WEST', 'Zip': '21218', 'Trade Name': 'KONG POCHA', 'Corp Name': 'SOY BEAN, INC.', 'License': 'LB'}, {'\ufeffBlock': '225-27', 'Street': '23RD STREET WEST', 'Address': '225-27 23RD STREET WEST', 'Zip': '21211', 'Trade Name': 'CLAVEL', 'Corp Name': 'HARLAN, INC', 'License': 'LB'}, {'\ufeffBlock': '400', 'Street': '23RD STREET WEST', 'Address': '400 23RD STREET WEST', 'Zip': '21211', 'Trade Name': 'W.C. HARLAN', 'Corp Name': '400 W. 23RD STREET, INC.', 'License': 'LD'}]


In [65]:
# Convert to DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

# Create full address
df['FullAddress'] = df['Address'] + ', Baltimore, MD, ' + df['Zip'] + ', USA'

# Display the DataFrame with the FullAddress column
print(df[['FullAddress']])

       ﻿Block               Street                      Address    Zip  \
0      16-Dec     20TH STREET WEST       12-16 20TH STREET WEST  21218   
1      225-27     23RD STREET WEST      225-27 23RD STREET WEST  21211   
2         400     23RD STREET WEST         400 23RD STREET WEST  21211   
3         101     25TH STREET EAST         101 25TH STREET EAST  21218   
4        1001     25TH STREET EAST        1001 25TH STREET EAST  21218   
...       ...                  ...                          ...    ...   
1168     6726    YOUNGSTOWN AVENUE       6726 YOUNGSTOWN AVENUE  21222   
1169  201-207  REDWOOD STREET EAST  201-207 REDWOOD STREET EAST  21202   
1170    4-Feb  PRESTON STREET EAST      2-4 PRESTON STREET EAST  21202   
1171      720     ALICEANNA STREET         720 ALICEANNA STREET  21202   
1172     1200    NORTH AVENUE WEST       1200 NORTH AVENUE WEST  21217   

                       Trade Name                     Corp Name License  
0                      KONG POCHA    

In [66]:
# Function to move directional indicators
def move_direction_to_start(address):
    # Pattern to match a directional indicator at the end of the street name
    pattern = re.compile(r'\b(North|South|East|West)\b', flags=re.IGNORECASE)
    
    # Check if there's a match for a directional indicator at the end
    match = pattern.search(address)
    if match:
        direction = match.group(0)
        # Remove directional indicator from the end
        address = pattern.sub('', address).strip()
        # Split address into parts at the first space
        parts = address.split(' ', 1)
        if len(parts) == 2:
            # Reconstruct address with direction after the first part (street number)
            address = f"{parts[0]} {direction} {parts[1]}"
        else:
            address = f"{address} {direction}"
    
    return address

# Function to remove second number in hyphenated address
def remove_second_number(address):
    return re.sub(r'(\d+)-\d+', r'\1', address, flags=re.IGNORECASE)

# Function to remove suite or unit information
def remove_suite_or_unit(address):
    return re.sub(r'\s+(Suite|Unit)\s+\d+', '', address, flags=re.IGNORECASE)

# Function to clean addresses
def clean_address(address):
    address = move_direction_to_start(address)
    address = remove_second_number(address)
    address = remove_suite_or_unit(address)
    return address

In [67]:
# Apply the cleaning function to the FullAddress column
df['CleanedAddress'] = df['FullAddress'].apply(clean_address)

# Print the cleaned addresses
print(df)

       ﻿Block               Street                      Address    Zip  \
0      16-Dec     20TH STREET WEST       12-16 20TH STREET WEST  21218   
1      225-27     23RD STREET WEST      225-27 23RD STREET WEST  21211   
2         400     23RD STREET WEST         400 23RD STREET WEST  21211   
3         101     25TH STREET EAST         101 25TH STREET EAST  21218   
4        1001     25TH STREET EAST        1001 25TH STREET EAST  21218   
...       ...                  ...                          ...    ...   
1168     6726    YOUNGSTOWN AVENUE       6726 YOUNGSTOWN AVENUE  21222   
1169  201-207  REDWOOD STREET EAST  201-207 REDWOOD STREET EAST  21202   
1170    4-Feb  PRESTON STREET EAST      2-4 PRESTON STREET EAST  21202   
1171      720     ALICEANNA STREET         720 ALICEANNA STREET  21202   
1172     1200    NORTH AVENUE WEST       1200 NORTH AVENUE WEST  21217   

                       Trade Name                     Corp Name License  \
0                      KONG POCHA   

In [8]:
# Example with progress tracking
total_addresses = len(df)
for i, address in enumerate(df['FullAddress']):
    df.at[i, 'CleanedAddress'] = clean_address(address)
    if i % 100 == 0:  # Print progress every 100 addresses
        print(f"Processed {i+1}/{total_addresses} addresses")

# Print the cleaned addresses
print(df)

Processed 1/1173 addresses
Processed 101/1173 addresses
Processed 201/1173 addresses
Processed 301/1173 addresses
Processed 401/1173 addresses
Processed 501/1173 addresses
Processed 601/1173 addresses
Processed 701/1173 addresses
Processed 801/1173 addresses
Processed 901/1173 addresses
Processed 1001/1173 addresses
Processed 1101/1173 addresses
       ﻿Block               Street                      Address    Zip  \
0      16-Dec     20TH STREET WEST       12-16 20TH STREET WEST  21218   
1      225-27     23RD STREET WEST      225-27 23RD STREET WEST  21211   
2         400     23RD STREET WEST         400 23RD STREET WEST  21211   
3         101     25TH STREET EAST         101 25TH STREET EAST  21218   
4        1001     25TH STREET EAST        1001 25TH STREET EAST  21218   
...       ...                  ...                          ...    ...   
1168     6726    YOUNGSTOWN AVENUE       6726 YOUNGSTOWN AVENUE  21222   
1169  201-207  REDWOOD STREET EAST  201-207 REDWOOD STREET E

In [68]:
# THESE ARE MANUAL CHANGES I SEE NEEDED TO OCCUR EVEN AFTER THE CLEANING
address_updates = {
    '1 NORTH HAVEN STREET  SUITE #101, Baltimore, MD, 21224, USA': '1 NORTH HAVEN STREET , Baltimore, MD, 21224, USA',
    '10 NORTH  AVENUE , Baltimore, MD, 21201, USA': '10 W NORTH  AVENUE , Baltimore, MD, 21201, USA',
    '1000 LANCASTER STREET, SUITE B, Baltimore, MD, 21202, USA': '1000 LANCASTER STREET, Baltimore, MD, 21202, USA',
    '1010 SOUTH OLDHAM STREET , Baltimore, MD, 21224, USA': '1010 OLDHAM STREET , Baltimore, MD, 21224, USA',
    '1030 WEST F-1 41ST STREET , Baltimore, MD, 21211, USA': '1030 WEST 41ST STREET , Baltimore, MD, 21211, USA',
    '1100 FLEET STREET #1302, Baltimore, MD, 21202, USA': '1100 FLEET STREET, Baltimore, MD, 21202, USA',
    '1100 WICOMICO STREET,A, Baltimore, MD, 21230, USA': '1100 WICOMICO STREET, Baltimore, MD, 21230, USA',
    '1101 NORTH RUSSELL STREET , Baltimore, MD, 21230, USA': '1101 RUSSELL STREET , Baltimore, MD, 21230, USA',
    '1148 NORTH  AVENUE , Baltimore, MD, 21202, USA': '1148 E NORTH  AVENUE , Baltimore, MD, 21202, USA',
    '#NAME?, Baltimore, MD, 21201, USA': '119 N Paca St, Baltimore MD 21201',
    '120 NORTH  AVENUE , Baltimore, MD, 21201, USA': '120 W NORTH  AVENUE , Baltimore, MD, 21201, USA',
    '1200 NORTH  AVENUE , Baltimore, MD, 21217, USA': '1200 W NORTH  AVENUE , Baltimore, MD, 21217, USA',
    '1209 NORTH  AVENUE , Baltimore, MD, 21217, USA': '1209 W NORTH  AVENUE , Baltimore, MD, 21217, USA',
    '1220 NORTH  AVENUE , Baltimore, MD, 21217, USA': '1220 W NORTH  AVENUE , Baltimore, MD, 21217, USA',
    '1301 SOUTH PONCA STREET , Baltimore, MD, 21224, USA': '1301 PONCA STREET , Baltimore, MD, 21224, USA',
    '1525 Russell St., 2105 Haines St. & 1555 Warner Street, Baltimore, MD, 21230, USA': '1525 Russell St, Baltimore, MD, 21230, USA',
    '1625 UNION AVENUE UNIT C, Baltimore, MD, 21211, USA': '1625 UNION AVENUE, Baltimore, MD, 21211, USA',
    '1641 ALICEANNA STREET #105, Baltimore, MD, 21231, USA': '1641 ALICEANNA STREET, Baltimore, MD, 21231, USA',
    '1700 UNION AVENUE SUITE D, Baltimore, MD, 21211, USA': '1700 UNION AVENUE, Baltimore, MD, 21211, USA',
    '1724 THAMES STREET & 808 S ANN STREET, Baltimore, MD, 21231, USA': '1724 THAMES STREET, Baltimore, MD, 21231, USA',
    '1725 SOUTH PONCA STREET , Baltimore, MD, 21224, USA': '1725 PONCA STREET , Baltimore, MD, 21224, USA',
    '1741 NORTH  AVENUE , Baltimore, MD, 21217, USA': '1741 W NORTH  AVENUE , Baltimore, MD, 21217, USA',
    '175 WEST OSTEND STREET  SUITE B-1, Baltimore, MD, 21230, USA': '175 WEST OSTEND STREET, Baltimore, MD, 21230, USA',
    '1801 NORTH  AVENUE , Baltimore, MD, 21213, USA': '1801 E NORTH  AVENUE , Baltimore, MD, 21213, USA',
    '1831 WEST MOSHER STREET , Baltimore, MD, 21217, USA': '1831 MOSHER STREET , Baltimore, MD, 21217, USA',
    '- HARBORPLACE-201 E.PRATT ST, Baltimore, MD, 21202, USA': '201 E. PRATT ST, Baltimore, MD, 21202, USA',
    '-- HARBORPLACE-201 E.PRATT ST, Baltimore, MD, 21202, USA': '201 E. PRATT ST, Baltimore, MD, 21202, USA',
    '2101 NORTH  AVENUE , Baltimore, MD, 21213, USA': '2101 E NORTH  AVENUE , Baltimore, MD, 21213, USA',
    '2115 NORTH  AVENUE , Baltimore, MD, 21217, USA': '2115 W NORTH  AVENUE , Baltimore, MD, 21217, USA',
    '213 SOUTH PENN STREET , Baltimore, MD, 21230, USA': '213 PENN STREET , Baltimore, MD, 21230, USA',
    '2135 NORTH  AVENUE , Baltimore, MD, 21213, USA': '2135 E NORTH  AVENUE , Baltimore, MD, 21213, USA',
    '2201 NORTH  AVENUE , Baltimore, MD, 21216, USA': '2201 W NORTH  AVENUE , Baltimore, MD, 21216, USA',
    '222 NORTH CHARLES STREET  UNIT B2, Baltimore, MD, 21201, USA': '222 NORTH CHARLES STREET , Baltimore, MD, 21201, USA',
    '222 ST. PAUL PL AND 225 N. CHARLES ST, Baltimore, MD, 21202, USA': '222 ST. PAUL PL, Baltimore, MD, 21202, USA',
    '2250 NORTH  AVENUE , Baltimore, MD, 21213, USA': '2250 E NORTH  AVENUE , Baltimore, MD, 21213, USA',
    '26 SOUTH ARLINGTON STREET , Baltimore, MD, 21223, USA': '26 S. ARLINGTON Ave, Baltimore, MD, 21223, USA',
    '2600 NORTH HOWARD STREET , SUITE #1000, Baltimore, MD, 21218, USA': '2600 NORTH HOWARD STREET, Baltimore, MD, 21218, USA',
    '2600 NORTH HOWARD STREET , SUITE #1100, Baltimore, MD, 21218, USA': '2600 NORTH HOWARD STREET, Baltimore, MD, 21218, USA',
    '2700 NORTH  AVENUE , Baltimore, MD, 21216, USA': '2700 W NORTH  AVENUE , Baltimore, MD, 21216, USA',
    '2863 NORTH  AVENUE , Baltimore, MD, 21216, USA': '2863 W NORTH  AVENUE , Baltimore, MD, 21216, USA',
    '3 NORTH  AVENUE , Baltimore, MD, 21202, USA': '3 E NORTH  AVENUE , Baltimore, MD, 21202, USA',
    '3 NORTH  AVENUE , Baltimore, MD, 21201, USA': '3 W NORTH  AVENUE , Baltimore, MD, 21201, USA',
    '30 LIGHT STREET, STE 2, Baltimore, MD, 21202, USA': '30 LIGHT STREET, Baltimore, MD, 21202, USA',
    '30 NORTH  AVENUE , Baltimore, MD, 21201, USA': '30 W NORTH  AVENUE , Baltimore, MD, 21201, USA',
    '- HARBORPLACE-301 LIGHT ST (1445), Baltimore, MD, 21202, USA': '301 LIGHT ST, Baltimore, MD, 21202, USA',
    '3107 NORTH  AVENUE , Baltimore, MD, 21216, USA': '3107 W NORTH  AVENUE , Baltimore, MD, 21216, USA',
    '329 WEST 1/2 29TH STREET , Baltimore, MD, 21211, USA': '329 WEST 29TH STREET , Baltimore, MD, 21211, USA',
    '33 NORTH  AVENUE , Baltimore, MD, 21201, USA': '33 W NORTH  AVENUE , Baltimore, MD, 21201, USA',
    '3300 CLIPPER MILL ROAD, STALL #7, Baltimore, MD, 21211, USA': '3300 CLIPPER MILL ROAD,, Baltimore, MD, 21211, USA',
    '335 NORTH  AVENUE , Baltimore, MD, 21202, USA': '335 E NORTH  AVENUE , Baltimore, MD, 21202, USA',
    '3800 EAST LOMBARD STREET , #13, Baltimore, MD, 21224, USA': '3800 EAST LOMBARD STREET , Baltimore, MD, 21224, USA',
    '3931 ERDMAN AVENUE (SHOP.CNTR), Baltimore, MD, 21213, USA': '3931 ERDMAN AVENUE, Baltimore, MD, 21213, USA',
    '401 SOUTH FURROW STREET , Baltimore, MD, 21223, USA': '401 FURROW STREET , Baltimore, MD, 21223, USA',
    '407 EAST PRESTON STREET , SUITE B, Baltimore, MD, 21202, USA': '407 EAST PRESTON STREET, Baltimore, MD, 21202, USA',
    '413 WEST COLDSPRING LANE  , Baltimore, MD, 21210, USA': '413 WEST COLD SPRING LANE  , Baltimore, MD, 21210, USA',
    '414 LIGHT STREET, SUITE103, Baltimore, MD, 21202, USA': '414 LIGHT STREET, Baltimore, MD, 21202, USA',
    '415 WEST COLDSPRING LANE , Baltimore, MD, 21210, USA': '415 WEST COLD SPRING LANE , Baltimore, MD, 21210, USA',
    '429 NORTH EUTAW STREET  1N, Baltimore, MD, 21201, USA': '429 NORTH EUTAW STREET, Baltimore, MD, 21201, USA',
    '4517A&B BOWLEYS LANE, Baltimore, MD, 21206, USA': '4517 BOWLEYS LANE, Baltimore, MD, 21206, USA',
    '4601 HILLEN RD-MORGAN STATE U., Baltimore, MD, 21251, USA': '4601 HILLEN RD., Baltimore, MD, 21251, USA',
    '4801 HARFORD ROAD, SUITE H1, Baltimore, MD, 21214, USA': '4801 HARFORD ROAD, Baltimore, MD, 21214, USA',
    '4801 HARFORD ROAD, SUITE S2, Baltimore, MD, 21214, USA': '4801 HARFORD ROAD, Baltimore, MD, 21214, USA',
    '507 WEST COLDSPRING LANE , Baltimore, MD, 21210, USA': '507 WEST COLD SPRING LANE , Baltimore, MD, 21210, USA',
    '5201 PARK HEIGHTS & BELVEDERE, Baltimore, MD, 21215, USA': '5201 PARK HEIGHTS Ave, Baltimore, MD, 21215, USA',
    '5402 A&B PARK HEIGHTS AVENUE, Baltimore, MD, 21215, USA': '5402 PARK HEIGHTS AVENUE, Baltimore, MD, 21215, USA',
    '5658 ALAMEDA, Baltimore, MD, 21239, USA': '5658 The ALAMEDA, Baltimore, MD, 21239, USA',
    '5601 EASTERN AVENUE K/A 5701A FLEET STREET, Baltimore, MD, 21224, USA': '5701 FLEET STREET, Baltimore, MD, 21224, USA',
    '5900 REISTERSTOWN ROAD, UNIT F, Baltimore, MD, 21215, USA': '5900 REISTERSTOWN ROAD, Baltimore, MD, 21215, USA',
    '600 EAST PRATT STREET  AKA 124 MARKET PLACE, Baltimore, MD, 21202, USA': '600 EAST PRATT STREET, Baltimore, MD, 21202, USA',
    '600 EAST PRATT STREET , SUITE # 105, Baltimore, MD, 21202, USA': '600 EAST PRATT STREET, Baltimore, MD, 21202, USA',
    '111 MARKET PLACE AKA 700 PRATT STREET E, Baltimore, MD, 21202, USA': '700 E PRATT STREET , Baltimore, MD, 21202, USA',
    '800 EAST ST. PAUL STREET AKA 24  MADISON STREET, Baltimore, MD, 21202, USA': '800 ST. PAUL STREET, Baltimore, MD, 21202, USA',
    '813 BROADWAYSOUTH, Baltimore, MD, 21231, USA': '813 S BROADWAY , Baltimore, MD, 21231, USA',
    '855 NORTH WOLFE STREET  SUITE D & E, Baltimore, MD, 21205, USA': '855 NORTH WOLFE STREET, Baltimore, MD, 21205, USA',
    '900 EAST FORT AVENUE  SUITE C, Baltimore, MD, 21230, USA': '900 EAST FORT AVENUE , Baltimore, MD, 21230, USA',
    '900 EAST FORT AVEUNE  ,, Baltimore, MD, 21230, USA': '900 EAST FORT AVENUE , Baltimore, MD, 21230, USA',
    '904 NORTH  AVENUE , Baltimore, MD, 21217, USA': '904 W NORTH  AVENUE , Baltimore, MD, 21217, USA',
}

# Initialize a list to track unchanged addresses
unchanged_addresses = []

# Update specific records based on old address
for old_address, new_address in address_updates.items():
    mask = df['CleanedAddress'] == old_address
    if not df.loc[mask].empty:
        df.loc[mask, 'CleanedAddress'] = new_address
    else:
        unchanged_addresses.append(old_address)

# Print the updated DataFrame
print("\nUpdated DataFrame:")
print(df['CleanedAddress'])

# Print unchanged addresses
print("\nUnchanged Addresses:")
for address in unchanged_addresses:
    print(address)


Updated DataFrame:
0         12 WEST 20TH STREET , Baltimore, MD, 21218, USA
1        225 WEST 23RD STREET , Baltimore, MD, 21211, USA
2        400 WEST 23RD STREET , Baltimore, MD, 21211, USA
3        101 EAST 25TH STREET , Baltimore, MD, 21218, USA
4       1001 EAST 25TH STREET , Baltimore, MD, 21218, USA
                              ...                        
1168    6726 YOUNGSTOWN AVENUE, Baltimore, MD, 21222, USA
1169    201 EAST REDWOOD STREET , Baltimore, MD, 21202...
1170    2 EAST PRESTON STREET , Baltimore, MD, 21202, USA
1171      720 ALICEANNA STREET, Baltimore, MD, 21202, USA
1172     1200 W NORTH  AVENUE , Baltimore, MD, 21217, USA
Name: CleanedAddress, Length: 1173, dtype: object

Unchanged Addresses:


In [75]:
# Initialize Nominatim geocoder with a unique user-agent
geolocator = Nominatim(user_agent="baltimore_alcohol")

# List to store geocoded locations
locations = []

# List to store addresses that failed to geocode
failed_addresses = []

# Function to handle geocoding with retries
def geocode_address(address, retries=3):
    try:
        location = geolocator.geocode(address, timeout=10)
        return location
    except (GeocoderTimedOut, GeocoderServiceError) as e:
        if retries > 0:
            time.sleep(1)  # Wait before retrying
            return geocode_address(address, retries - 1)
        else:
            print(f"Error geocoding {address}: {e}")
            return None

# Process addresses in batches
batch_size = 100
for i in range(0, len(df), batch_size):
    batch = df['CleanedAddress'][i:i + batch_size]
    for address in batch:
        location = geocode_address(address)
        if location:
            locations.append((address, location.latitude, location.longitude))
        else:
            failed_addresses.append(address)
    print(f"Processed batch {i // batch_size + 1}")
    time.sleep(5)  # Delay between batches to respect rate limits

# Print geocoded locations
for address, lat, lon in locations[:10]:
    print(f"Address: {address}\nLatitude: {lat}\nLongitude: {lon}\n")

# Print count of failed addresses
print(f"\nNumber of addresses not geocoded: {len(failed_addresses)}")
print("Failed Addresses:")
for address in failed_addresses:
    print(address)

Processed batch 1
Processed batch 2
Processed batch 3
Processed batch 4
Processed batch 5
Processed batch 6
Processed batch 7
Processed batch 8
Processed batch 9
Processed batch 10
Processed batch 11
Processed batch 12
Address: 12 WEST 20TH STREET , Baltimore, MD, 21218, USA
Latitude: 39.3123298
Longitude: -76.6171821

Address: 225 WEST 23RD STREET , Baltimore, MD, 21211, USA
Latitude: 39.3149871
Longitude: -76.6204196

Address: 400 WEST 23RD STREET , Baltimore, MD, 21211, USA
Latitude: 39.31520935
Longitude: -76.62208054270036

Address: 101 EAST 25TH STREET , Baltimore, MD, 21218, USA
Latitude: 39.317737
Longitude: -76.6152268

Address: 1001 EAST 25TH STREET , Baltimore, MD, 21218, USA
Latitude: 39.31748465
Longitude: -76.59994425053124

Address: 120 WEST 25TH STREET , Baltimore, MD, 21218, USA
Latitude: 39.3179679
Longitude: -76.61915859267302

Address: 134 WEST 25TH STREET , Baltimore, MD, 21218, USA
Latitude: 39.317950800000006
Longitude: -76.6194893473623

Address: 127 WEST 27TH S

In [76]:
# Convert locations list to DataFrame
df_locations = pd.DataFrame(locations, columns=['CleanedAddress', 'Latitude', 'Longitude'])

# Step 3: Merge DataFrames based on FullAddress
df_merged = pd.merge(df, df_locations, on='CleanedAddress', how='left')

# Step 4: Save the merged DataFrame back to a new CSV file
df_merged.to_csv('merged_with_locations.csv', index=False)

# Print the merged DataFrame for verification
print(df_merged)

       ﻿Block               Street                      Address    Zip  \
0      16-Dec     20TH STREET WEST       12-16 20TH STREET WEST  21218   
1      225-27     23RD STREET WEST      225-27 23RD STREET WEST  21211   
2         400     23RD STREET WEST         400 23RD STREET WEST  21211   
3         101     25TH STREET EAST         101 25TH STREET EAST  21218   
4        1001     25TH STREET EAST        1001 25TH STREET EAST  21218   
...       ...                  ...                          ...    ...   
1240     6726    YOUNGSTOWN AVENUE       6726 YOUNGSTOWN AVENUE  21222   
1241  201-207  REDWOOD STREET EAST  201-207 REDWOOD STREET EAST  21202   
1242    4-Feb  PRESTON STREET EAST      2-4 PRESTON STREET EAST  21202   
1243      720     ALICEANNA STREET         720 ALICEANNA STREET  21202   
1244     1200    NORTH AVENUE WEST       1200 NORTH AVENUE WEST  21217   

                       Trade Name                     Corp Name License  \
0                      KONG POCHA   

In [77]:
# Drop duplicates based on Address and Trade Name
df_unique = df_merged.drop_duplicates(subset=['Address', 'Trade Name'])
len(df_unique)

1172

In [89]:
# List of addresses of outlets to mark as 'Nonconforming', per BLCC
nonconforming_addresses = [
    '26 SOUTH CATHERINE STREET , Baltimore, MD, 21223, USA',
    '401 FURROW STREET , Baltimore, MD, 21223, USA',
    '255 NORTH PAYSON STREET , Baltimore, MD, 21223, USA',
    '1800 ROSEDALE STREET, Baltimore, MD, 21216, USA',
    '1200 W NORTH  AVENUE , Baltimore, MD, 21217, USA',
    '2100 EAST BIDDLE STREET , Baltimore, MD, 21213, USA',
    '1700 CLIFTVIEW AVENUE, Baltimore, MD, 21213, USA',
    '1800 DIVISION STREET, Baltimore, MD, 21217, USA',
    '3401 EDMONDSON AVENUE, Baltimore, MD, 21229, USA',
    '3859 ELMLEY AVENUE, Baltimore, MD, 21213, USA',
    '4742 FREDERICK AVENUE, Baltimore, MD, 21229, USA',
    '1501 NORTH FULTON AVENUE , Baltimore, MD, 21217, USA',
    '3600 WEST GARRISON AVENUE , Baltimore, MD, 21215, USA',
    '300 NORTH GILMOR STREET , Baltimore, MD, 21223, USA',
    '1655 NORTH MONROE STREET , Baltimore, MD, 21217, USA',
    '1831 MOSHER STREET , Baltimore, MD, 21217, USA',
    '3927 PARK HEIGHTS AVENUE, Baltimore, MD, 21215, USA',
]

# Add a new column 'BLCC_nonconform' and mark rows accordingly
df_unique['BLCC_nonconform'] = 0  # Default value

# Mark nonconforming addresses using .loc to avoid SettingWithCopyWarning
df_unique.loc[df_unique['CleanedAddress'].isin(nonconforming_addresses), 'BLCC_nonconform'] = 1

# Display the updated DataFrame
print(df_unique)

       ﻿Block               Street                      Address    Zip  \
0      16-Dec     20TH STREET WEST       12-16 20TH STREET WEST  21218   
1      225-27     23RD STREET WEST      225-27 23RD STREET WEST  21211   
2         400     23RD STREET WEST         400 23RD STREET WEST  21211   
3         101     25TH STREET EAST         101 25TH STREET EAST  21218   
4        1001     25TH STREET EAST        1001 25TH STREET EAST  21218   
...       ...                  ...                          ...    ...   
1240     6726    YOUNGSTOWN AVENUE       6726 YOUNGSTOWN AVENUE  21222   
1241  201-207  REDWOOD STREET EAST  201-207 REDWOOD STREET EAST  21202   
1242    4-Feb  PRESTON STREET EAST      2-4 PRESTON STREET EAST  21202   
1243      720     ALICEANNA STREET         720 ALICEANNA STREET  21202   
1244     1200    NORTH AVENUE WEST       1200 NORTH AVENUE WEST  21217   

                       Trade Name                     Corp Name License  \
0                      KONG POCHA   

/var/folders/16/08fw8cvd0g7495d8392p8vk40000gn/T/ipykernel_665/2661230699.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique['BLCC_nonconform'] = 0  # Default value


In [90]:
df_unique['BLCC_nonconform'].value_counts()

BLCC_nonconform
0    1155
1      17
Name: count, dtype: int64

In [91]:
# # Remove unwanted columns: 'Block', 'Street', 'Address', 'Zip'
# columns_to_drop = ['Block', 'Street', 'Address', 'Zip']
# df_unique.drop(columns_to_drop, axis=1, inplace=True)

# Save the DataFrame to a new CSV file
df_unique.to_csv('geocoded_unique.csv', index=False)

In [86]:
df_unique.columns

Index(['﻿Block', 'Street', 'Address', 'Zip', 'Trade Name', 'Corp Name',
       'License', 'FullAddress', 'CleanedAddress', 'Latitude', 'Longitude',
       'BLCC_nonconform'],
      dtype='object')